In [16]:
from dotenv import load_dotenv
import os 
from groq import Groq
from openai import OpenAI
import pandas as pd

_ = load_dotenv()

GROQ_API_KEY = os.getenv('GROQ_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client_groq = Groq(
    api_key=GROQ_API_KEY,
)

client_openai = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [25]:
def chat_completion(client, content, model: str = "llama3-70b-8192"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant. Generate an article based on the user's request."},
            {"role": "user", "content": content}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "article_response",
                "schema": {
                    "type": "object",
                    "properties": {
                        "article_title": {"type": "string"},
                        "sections": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "headline": {"type": "string"},
                                    "body": {"type": "string"}
                                },
                                "required": ["headline", "body"],
                                "additionalProperties": False
                            }
                        }
                    },
                    "required": ["article_title", "sections"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )

    return response.choices[0].message.content

In [9]:
df = pd.read_csv('../src/dataset/arduino_seo_keywords.csv')

df.head()

,Main Keyword,Secondary Keywords
0,Learning Embedded Systems,"Arduino programming, microcontroller basics, r..."
1,Arduino Projects,"Arduino beginner projects, LED blink project, ..."
2,Arduino for Beginners,"Getting started with Arduino, Arduino starter ..."


In [ ]:
content = "Generate a 200 words article about Learning Embedded Systems with Arduino." 

article = chat_completion(client_openai, content, model="gpt-4o")

In [33]:
import json

article = json.loads(article)

In [34]:

with open('article_output.json', 'w') as file:
    json.dump(article, file, indent=2)